# Robot Comm

In [90]:
%register 'serial:///dev/serial0'
%connect robot-stm

Connected to robot-stm @ serial:///dev/serial0


In [120]:
%register 'serial:///dev/serial0'
%connect robot-stm
%rsync
%softreset

# upload program

from micropython import const
from pyb import UART, Timer, LED
from array import array
import gc, micropython, time

micropython.alloc_emergency_exception_buf(100)

def main():
    from pyb import UART
    uart3 = UART(3, 1000000, timeout=500)
    micropython.kbd_intr(-1)
    status = array('I', [7, 2, 9])
    speed = array('I', [1,2])
    while True:
        if uart3.any():
            tc = uart3.read(1)
            if tc == b'I':
                uart3.write(b'1')
                uart3.write(array('I', [7, 20000]))
            elif tc == b'f':
                uart3.write(b'2')
                uart3.write(array('f', [3, 3.21]))
            elif tc == b'v':
                uart3.write(b'3')
                uart3.readinto(status)
                uart3.write(status)

        
%%host

from iot_device.pydevice import Pydevice
from serial import Serial
from array import array
import time

print("start MCU ...")

try:

    # connect to repl & start program
    with Serial('/dev/serial0', 115200, parity='N',
                    timeout=0.5,
                    write_timeout=2,
                    exclusive= True
                ) as serial:
        pyb = Pydevice(serial)
        pyb.enter_raw_repl()
        pyb.exec('import main_')
        
        print("main() ...")
        # pyb.exec_raw_no_follow('main()')
        pyb.exec_raw_no_follow('main_.XXX();')


        print("switching to 1MBaud")
        
    time.sleep(1)

    # connect to repl & start program
    with Serial('/dev/serial0', 1000000, parity='N',
                    timeout=1,
                    write_timeout=2,
                    exclusive= True
                ) as serial:
        while True:
            serial.write(b'I')
            while serial.in_waiting == 0:
                print("waiting")
                time.sleep(1)
                
            type_code = serial.read(1)
            print("type", type_code)
            print("data", array('I', serial.read(8)))
            
            serial.write(b'f')
            type_code = serial.read(1)
            print("type", type_code)
            print("data", array('f', serial.read(8)))
            
            serial.write(b'v')
            serial.write(array('I', [1,3,57]))
            type_code = serial.read(1)
            print("type", type_code)
            print("data", array('I', serial.read(12)))
            
            time.sleep(0.1)
            if serial.in_waiting:
                print("tail", serial.read(serial.in_waiting))
            break

finally:

    print("reset MCU ...")

    from gpiozero import LED
    from time import sleep

    with LED(21) as nrst:
        nrst.off()
        sleep(0.1)
        nrst.on()
        sleep(0.1)


Connected to robot-stm @ serial:///dev/serial0
Directories match

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

start MCU ...
main() ...
switching to 1MBaud
waiting
type b'1'
data array('I', [7, 20000])
type b'2'
data array('f', [3.0, 3.2100000381469727])
type b'3'
data array('I', [1, 3, 57])
reset MCU ...
